In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# --- Define Parameters ---
IMG_SIZE = 160
BATCH_SIZE = 32
DATA_DIR = "Model_Dataset" # <--- Make sure this path is correct
NUM_CLASSES = 6 # Set to the number of folders you have

# --- Load and Prepare the Data ---
train_dataset = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

# Store class names for later use in predictions
class_names = train_dataset.class_names
print("Found classes:", class_names)

Found 57946 files belonging to 6 classes.
Using 46357 files for training.
Found 57946 files belonging to 6 classes.
Using 11589 files for validation.
Found classes: ['Broken Road', 'Broken Traffic Light', 'Drainage', 'Garbage', 'Non-Civic', 'Potholes']


In [3]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# 1. Create a data augmentation block
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.2),
  layers.RandomZoom(0.2),
])

# 2. Load the pre-trained MobileNetV2 base
base_model = MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False # Freeze the base

# 3. Get the input preprocessing function
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# 4. Build the final model
model = models.Sequential([
    tf.keras.Input(shape=IMG_SHAPE),
    data_augmentation,
    # THE FIX IS HERE: Wrap the function in a Lambda layer
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

In [4]:
# Compile the model with the new loss function
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    # IMPORTANT: Changed for multi-class
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

print("--- Model Summary Before Fine-Tuning ---")
model.summary()

# Run the initial training
print("\n--- Starting Initial Training ---")
initial_epochs = 10
history = model.fit(
    train_dataset,
    epochs=initial_epochs,
    validation_data=validation_dataset
)

--- Model Summary Before Fine-Tuning ---


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_160            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │         7,686 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,265,670 (8.64 MB)

 Trainable params: 7,686 (30.02 KB)

 Non-trainable params: 2,257,984 (8.61 MB)


--- Starting Initial Training ---
Epoch 1/10


  13/1449 ━━━━━━━━━━━━━━━━━━━━ 8:21 349ms/step - accuracy: 0.0481 - loss: 3.2224

KeyboardInterrupt: 

In [9]:
print("\n--- Starting Fine-Tuning ---")
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Re-compile the model with a very low learning rate
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00001),
    metrics=['accuracy']
)
print("\n--- Model Summary After Unfreezing ---")
model.summary()

# Continue training the model
fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

history_fine = model.fit(
    train_dataset,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=validation_dataset
)

# Save the final, most powerful model
model.save('civic_issue_classifier.keras')
print("\n--- Model fine-tuning complete and saved as civic_issue_classifier.keras ---")


--- Starting Fine-Tuning ---

--- Model Summary After Unfreezing ---


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_160            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │         7,686 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,265,670 (8.64 MB)

 Trainable params: 1,869,126 (7.13 MB)

 Non-trainable params: 396,544 (1.51 MB)

Epoch 10/20
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 636s 435ms/step - accuracy: 0.9578 - loss: 0.1477 - val_accuracy: 0.9957 - val_loss: 0.0146
Epoch 11/20
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 627s 433ms/step - accuracy: 0.9938 - loss: 0.0196 - val_accuracy: 0.9965 - val_loss: 0.0121
Epoch 12/20
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 627s 433ms/step - accuracy: 0.9954 - loss: 0.0147 - val_accuracy: 0.9967 - val_loss: 0.0115
Epoch 13/20
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 627s 432ms/step - accuracy: 0.9964 - loss: 0.0117 - val_accuracy: 0.9969 - val_loss: 0.0108
Epoch 14/20
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 629s 434ms/step - accuracy: 0.9963 - loss: 0.0107 - val_accuracy: 0.9967 - val_loss: 0.0104
Epoch 15/20
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 628s 433ms/step - accuracy: 0.9971 - loss: 0.0090 - val_accuracy: 0.9963 - val_loss: 0.0101
Epoch 16/20
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 630s 435ms/step - accuracy: 0.9976 - loss: 0.0074 - val_accuracy: 0.9973 - val_loss: 0.0082
Epoch 17/20
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 627s 433ms/s

In [20]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

IMG_SIZE = 160
NUM_CLASSES = 6
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
class_names = ['Broken Road', 'Broken Street Light', 'Drainage', 'Garbage', 'Non-Civic', 'Potholes']

base_model = models.Sequential([
    tf.keras.Input(shape=IMG_SHAPE),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.Lambda(preprocess_input),
    MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet'),
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

print("Loading Weights")
base_model.load_weights('civic_issue_classifier.keras')
print("Weights loaded successfully!")


image_path = 'img5.png' 


img = tf.keras.utils.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = tf.keras.utils.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)


predictions = base_model.predict(img_batch)
score = tf.nn.softmax(predictions[0])


predicted_class_index = np.argmax(score)
predicted_class_name = class_names[predicted_class_index]
confidence = 100 * np.max(score)

print(f"\n'{predicted_class_name}' with a {confidence:.2f}% confidence.")

Loading Weights
Weights loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

'Broken Road' with a 35.22% confidence.
